<a href="https://colab.research.google.com/github/paularedondosastre/UFV-visualization/blob/main/Ejercicios_clase/Visualizaci%C3%B3n_clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Subir archivo desde computadora
uploaded = files.upload()

Saving datos_ejercicio_ventas.csv to datos_ejercicio_ventas.csv


In [4]:
df = pd.read_csv('datos_ejercicio_ventas.csv')

# Ver las primeras filas del DataFrame
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


# Un primer análisis exploratorio

In [5]:
num_filas = df.shape[0]
print(f'El archivo tiene {num_filas} filas.')

El archivo tiene 18666 filas.


Ver el número de actuals y de forecats.

In [6]:
conteo_scenarios = df['SCENARIO'].value_counts()
# Mostrar los resultados
print(conteo_scenarios)

SCENARIO
AI_forecast    17766
actual           900
Name: count, dtype: int64


Como nos ha dicho que el horizonte de previsión es el mismo para todos los productos, vamos a buscar cual es el horizonte de predicción para lipton en portugal. Lo que hay que hacer es fijar el country, subbrand, forecast y forecast_year.

In [7]:
# Filtrar filas donde el país sea "Portugal" y la subbrand sea "Lipton"
df_filtrado = df[(df['COUNTRY'] == 'Portugal') & (df['SUBBRAND'] == 'Lipton (L3)')]

# Mostrar las primeras filas del DataFrame filtrado
df_filtrado.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,7.543562e+05
292,Portugal,Lipton (L3),2023,10,actual,NaN,NaN,1.206886e+06
454,Portugal,Lipton (L3),2024,9,AI_forecast,AI_P04F,2023.0,7.122028e+05
490,Portugal,Lipton (L3),2023,7,AI_forecast,AI_P04F,2023.0,8.897866e+05
530,Portugal,Lipton (L3),2023,9,AI_forecast,AI_P06F,2023.0,7.676745e+05


In [8]:
df_filtrado_fijo = df_filtrado[(df_filtrado['FORECAST'] == "AI_P02F") & (df_filtrado['FORECAST_YEAR'] == 2023)]
meses = df_filtrado_fijo['FORECAST'].count()  # Contamos los valores no nulos en la columna 'FORECAST'
# Mostrar el número de filas
print(f'El horizonte temporal es {meses} meses.')

El horizonte temporal es 18 meses.


# Pregunta 1
Como se distribuyen las ventas realizadas en:

• Cada país

• Cada mes y año

• Cada marca

Ventas por país

In [10]:
# Filtrar el DataFrame para incluir solo los datos reales ('actual')
df_actual = df[df['SCENARIO'] == 'actual']

In [15]:
import plotly.express as px
df_grouped = df_actual.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Distribución de ventas por país usando solo datos reales
fig = px.bar(df_grouped, x='COUNTRY', y='AMOUNT', color='COUNTRY', title="Distribución de Ventas por País",
             labels={'AMOUNT': 'Cantidad de Productos Vendidos', 'COUNTRY': 'País'})
fig.update_layout(xaxis_title="País", yaxis_title="Cantidad de Productos Vendidos")
fig.show()

In [16]:
import plotly.express as px

# Filtrar el DataFrame para incluir solo los datos reales ('actual')
df_actual = df[df['SCENARIO'] == 'actual']

# Agrupar los datos por país
df_grouped = df_actual.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Gráfico de Donut (anillo)
fig = px.pie(df_grouped, names='COUNTRY', values='AMOUNT', title="Distribución de Ventas por País",
             hole=0.3,  # Crea el agujero central para el efecto donut
             labels={'AMOUNT': 'Cantidad de Productos Vendidos', 'COUNTRY': 'País'})

# Asegurarnos de que todas las categorías tienen el mismo "pull"
# Usamos un "pull" pequeño para todos, pero ajustamos visualmente el gráfico con el parámetro `direction` para separar uniformemente.
fig.update_traces(pull=[0.1]*len(df_grouped), textinfo='percent+label')

# Ajustamos la rotación para que las porciones grandes también se separen igual
fig.update_traces(rotation=90)  # Rotación inicial para dar espacio entre las categorías

# Mostrar el gráfico
fig.update_layout(title_x=0.5)  # Centrar el título

fig.show()




Ventas por mes y año

In [17]:
# Crear una copia de `df_actual` para trabajar sin la advertencia
df_actual_copia = df_actual.copy()

# Crear la columna `DATE` en la copia del DataFrame para la agrupación por mes y año
df_actual_copia['DATE'] = pd.to_datetime(df_actual_copia[['YEAR', 'MONTH']].assign(day=1))

# Agrupar ventas por fecha
ventas_por_mes_año = df_actual_copia.groupby('DATE')['AMOUNT'].sum().reset_index()

# Gráfico de línea de ventas por mes y año
import plotly.express as px

fig = px.line(ventas_por_mes_año, x='DATE', y='AMOUNT', title='Distribución de Ventas por Mes y Año',
              labels={'AMOUNT': 'Cantidad de Productos Vendidos', 'DATE': 'Fecha'})
fig.update_traces(mode='lines+markers')
fig.show()



In [18]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Agrupar por mes, año y marca (SUBBRAND)
df_actual_copia['DATE'] = pd.to_datetime(df_actual_copia[['YEAR', 'MONTH']].assign(day=1))
ventas_por_mes_año_marca = df_actual_copia.groupby(['DATE', 'SUBBRAND'])['AMOUNT'].sum().reset_index()

# Sumar las ventas totales por mes (sin tener en cuenta la marca)
ventas_por_mes_año_total = df_actual_copia.groupby('DATE')['AMOUNT'].sum().reset_index()

# Crear el gráfico de barras apiladas con la paleta de colores por defecto
fig = px.bar(ventas_por_mes_año_marca, x='DATE', y='AMOUNT', color='SUBBRAND',
             title='Distribución de Ventas por Marca y Mes',
             labels={'AMOUNT': 'Cantidad de Productos Vendidos', 'DATE': 'Fecha'},
             height=600, barmode='stack', opacity=0.8)  # Barras más transparentes con opacidad 0.5

# Añadir la línea que muestra la distribución total de ventas con color negro opaco
fig.add_trace(go.Scatter(x=ventas_por_mes_año_total['DATE'],
                         y=ventas_por_mes_año_total['AMOUNT'],
                         mode='lines',
                         name='Ventas Totales',
                         line=dict(color='black', width=2)))  # Línea negra y opaca

# Actualizar la apariencia del gráfico
fig.update_layout(
    xaxis_title="Fecha",
    yaxis_title="Cantidad de Productos Vendidos",
    title_x=0.5,
    xaxis=dict(tickformat="%b %Y", tickangle=45),
    showlegend=True  # Mostrar leyenda para la línea de ventas totales
)

# Mostrar el gráfico
fig.show()


In [37]:
import plotly.graph_objects as go
import pandas as pd
import plotly.subplots as sp

# Agrupar las ventas por mes y marca
ventas_por_mes_marca = df_actual_copia.groupby(['YEAR', 'MONTH', 'SUBBRAND'])['AMOUNT'].sum().reset_index()

# Crear una columna de fecha
ventas_por_mes_marca['DATE'] = pd.to_datetime(ventas_por_mes_marca[['YEAR', 'MONTH']].assign(day=1))

# Obtener el número de marcas y calcular el número de filas y columnas para el grid
num_marcas = len(ventas_por_mes_marca['SUBBRAND'].unique())
ncols = 3  # Número de columnas (3 por fila)
nrows = (num_marcas + ncols - 1) // ncols  # Número de filas, ajustado para distribuir bien las marcas

# Crear la figura con subgráficas
fig = sp.make_subplots(rows=nrows, cols=ncols,
                       subplot_titles=ventas_por_mes_marca['SUBBRAND'].unique(),
                       shared_xaxes=False, shared_yaxes=False,  # No compartir los ejes
                       vertical_spacing=0.25, horizontal_spacing=0.10)  # Aumentar el espacio vertical

# Iterar sobre cada marca y añadir el gráfico correspondiente a cada subgráfico
for i, marca in enumerate(ventas_por_mes_marca['SUBBRAND'].unique()):
    # Filtrar los datos para la marca actual
    df_marca = ventas_por_mes_marca[ventas_por_mes_marca['SUBBRAND'] == marca]

    # Determinar la posición de la subgráfica (fila, columna)
    row = i // ncols + 1
    col = i % ncols + 1

    # Añadir el gráfico de líneas a la subgráfica correspondiente
    fig.add_trace(go.Scatter(x=df_marca['DATE'], y=df_marca['AMOUNT'], mode='lines', name=marca),
                  row=row, col=col)

# Actualizar la apariencia general del gráfico
fig.update_layout(
    height=300 * nrows,  # Ajustar la altura de la figura para hacerla más pequeña
    width=1500,  # Aumentar el ancho de la figura para darle más espacio horizontal
    title_text='Distribución de Ventas Mensuales por Marca',
    showlegend=False,
    title_x=0.5,
    xaxis_title="Fecha",
    yaxis_title="Ventas Totales",
)

# Ajustar los ejes de cada subgráfico para que no se sobrepongan y poner los números del eje X en horizontal
fig.update_xaxes(tickformat="%b %Y", tickangle=0)  # Establecer los números del eje X en horizontal

# Mostrar el gráfico
fig.show()



Ventas por Marca

In [38]:
# Distribución de ventas por marca usando solo datos reales
df_2 = df_actual.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

fig = px.bar(df_2, x='SUBBRAND', y='AMOUNT', color='SUBBRAND', title="Distribución de Ventas por Marca",
             labels={'AMOUNT': 'Cantidad de Productos Vendidos', 'SUBBRAND': 'Marca'})
fig.update_layout(xaxis_title="Marca", yaxis_title="Cantidad de Productos Vendidos")
fig.show()


In [39]:
import plotly.express as px

# Agrupar y sumar las ventas por 'SUBBRAND'
df_2 = df_actual.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

# Crear gráfico de treemap con colores automáticos para cada marca
fig = px.treemap(df_2,
                 path=['SUBBRAND'],    # Define la jerarquía, en este caso sólo 'SUBBRAND'
                 values='AMOUNT',      # Define los valores que determinarán el tamaño de los rectángulos
                 color='SUBBRAND',     # Asigna colores automáticos distintos a cada marca
                 title="Distribución de Ventas por Marca",
                 labels={'AMOUNT': 'Cantidad de Productos Vendidos', 'SUBBRAND': 'Marca'})

# Mostrar el gráfico
fig.show()




# Pregunta 2: Tendencia y Estacionalidad de Ventas


Cual es la tendencia y estacionalidad de todas las ventas del país con menos ventas.



In [16]:
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Obtener el país con menos ventas en los datos reales
pais_menos_ventas = df_actual.groupby('COUNTRY')['AMOUNT'].sum().idxmin()

# Filtrar las ventas para el país con menos ventas
ventas_pais_menos = df_actual[df_actual['COUNTRY'] == pais_menos_ventas].copy()

# Crear la columna `DATE` en el DataFrame para la agrupación por mes y año
ventas_pais_menos['DATE'] = pd.to_datetime(ventas_pais_menos[['YEAR', 'MONTH']].assign(day=1))

# Agrupar por fecha y sumar las ventas (AMOUNT) por cada fecha
ventas_por_mes = ventas_pais_menos.groupby('DATE')['AMOUNT'].sum().reset_index()

# Alinear las fechas para la regresión lineal (convertir la fecha en un número)
ventas_por_mes['Fecha_numerica'] = (ventas_por_mes['DATE'] - ventas_por_mes['DATE'].min()) / np.timedelta64(1, 'D')

# Ajustar una regresión lineal para la tendencia
model = LinearRegression()
model.fit(ventas_por_mes[['Fecha_numerica']], ventas_por_mes['AMOUNT'])
ventas_por_mes['Tendencia_Lineal'] = model.predict(ventas_por_mes[['Fecha_numerica']])

# Ajustar una tendencia polinómica de segundo orden
z = np.polyfit(range(len(ventas_por_mes)), ventas_por_mes['AMOUNT'], 2)
p = np.poly1d(z)
ventas_por_mes['Tendencia_Polinomica'] = p(range(len(ventas_por_mes)))

# Crear el gráfico base
fig = px.line(title=f'Tendencia de Ventas en {pais_menos_ventas} (Datos Reales)',
              labels={'AMOUNT': 'Cantidad de Productos Vendidos', 'DATE': 'Fecha'})

# Añadir los datos reales con la leyenda "Datos Actuales"
fig.add_scatter(x=ventas_por_mes['DATE'], y=ventas_por_mes['AMOUNT'],
                mode='lines+markers', name='Datos Actuales', line=dict(color='blue'))

# Añadir la tendencia lineal (regresión)
fig.add_scatter(x=ventas_por_mes['DATE'], y=ventas_por_mes['Tendencia_Lineal'],
                mode='lines', name='Tendencia Lineal (Regresión)', line=dict(color='red'))

# Añadir la tendencia polinómica
fig.add_scatter(x=ventas_por_mes['DATE'], y=ventas_por_mes['Tendencia_Polinomica'],
                mode='lines', name='Tendencia Polinómica', line=dict(color='green', dash='dot'))

# Mostrar el gráfico
fig.update_layout(title=f'Tendencia de Ventas en {pais_menos_ventas}',
                  xaxis_title='Fecha', yaxis_title='Cantidad de Productos Vendidos')
fig.show()


El análisis de la tendencia muestra un ligero aumento en las ventas, y tanto la regresión lineal como la tendencia polinómica dan resultados muy parecidos. Sin embargo, esta subida es bastante suave, y como solo estamos considerando un periodo de poco más de un año, no podemos decir con seguridad que haya una tendencia significativa a largo plazo.

Vamos a calcular el promedio de ventas por mes para ver si hay patrones estacionales a lo largo del tiempo. Así, podemos identificar meses con ventas consistentemente altas o bajas, resaltando patrones como picos en verano o caídas en otras épocas. El gráfico de barras visualiza esta variación mensual, revelando estacionalidad en caso de que existan esos picos o bajas recurrentes.

In [17]:
import plotly.express as px
import pandas as pd

# Extraer el mes y calcular el promedio por cada mes
ventas_por_mes['Mes'] = ventas_por_mes['DATE'].dt.month
ventas_estacionalidad = ventas_por_mes.groupby('Mes')['AMOUNT'].mean().reset_index()

# Crear gráfico de barras para estacionalidad
fig = px.bar(ventas_estacionalidad, x='Mes', y='AMOUNT',
             title=f'Estacionalidad de Ventas en {pais_menos_ventas}',
             labels={'AMOUNT': 'Cantidad Promedio de Productos Vendidos', 'Mes': 'Mes'})

# Añadir una línea para conectar los puntos de cada mes, resaltando estacionalidad
fig.add_scatter(x=ventas_estacionalidad['Mes'], y=ventas_estacionalidad['AMOUNT'],
                mode='lines+markers', name='Promedio Mensual', line=dict(color='blue'))

# Actualizar títulos de los ejes
fig.update_layout(xaxis_title="Mes", yaxis_title="Ventas Promedio")

# Mostrar gráfico
fig.show()



Observamos que no existe un patrón estacional fuerte, ya que las ventas mensuales permanecen relativamente constantes con variaciones menores e inconsistentes a lo largo del año. Sin embargo, se identifica un aumento leve en los meses de julio y agosto. Esta tendencia puede estar relacionada con la temporada de verano, cuando es más común el consumo de bebidas como Lipton y Pepsi ya que son refrescantes para las épocas de calor.

Para el análisis de la marca con más ventas

In [18]:
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Obtener la marca con más ventas en los datos reales
marca_mas_ventas = df_actual.groupby('SUBBRAND')['AMOUNT'].sum().idxmax()

# Filtrar las ventas para la marca con más ventas
ventas_marca_mas = df_actual[df_actual['SUBBRAND'] == marca_mas_ventas].copy()

# Crear la columna `DATE` en el DataFrame para la agrupación por mes y año
ventas_marca_mas['DATE'] = pd.to_datetime(ventas_marca_mas[['YEAR', 'MONTH']].assign(day=1))

# Agrupar por fecha y sumar las ventas (AMOUNT) por cada fecha
ventas_por_mes_marca = ventas_marca_mas.groupby('DATE')['AMOUNT'].sum().reset_index()

# Alinear las fechas para la regresión lineal (convertir la fecha en un número)
ventas_por_mes_marca['Fecha_numerica'] = (ventas_por_mes_marca['DATE'] - ventas_por_mes_marca['DATE'].min()) / np.timedelta64(1, 'D')

# Ajustar una regresión lineal para la tendencia
model = LinearRegression()
model.fit(ventas_por_mes_marca[['Fecha_numerica']], ventas_por_mes_marca['AMOUNT'])
ventas_por_mes_marca['Tendencia_Lineal'] = model.predict(ventas_por_mes_marca[['Fecha_numerica']])

# Ajustar una tendencia polinómica de segundo orden
z = np.polyfit(range(len(ventas_por_mes_marca)), ventas_por_mes_marca['AMOUNT'], 2)
p = np.poly1d(z)
ventas_por_mes_marca['Tendencia_Polinomica'] = p(range(len(ventas_por_mes_marca)))

# Crear el gráfico base
fig = px.line(title=f'Tendencia de Ventas en {marca_mas_ventas} (Datos Reales)',
              labels={'AMOUNT': 'Cantidad de Productos Vendidos', 'DATE': 'Fecha'})

# Añadir los datos reales con la leyenda "Datos Actuales"
fig.add_scatter(x=ventas_por_mes_marca['DATE'], y=ventas_por_mes_marca['AMOUNT'],
                mode='lines+markers', name='Datos Actuales', line=dict(color='blue'))

# Añadir la tendencia lineal (regresión)
fig.add_scatter(x=ventas_por_mes_marca['DATE'], y=ventas_por_mes_marca['Tendencia_Lineal'],
                mode='lines', name='Tendencia Lineal (Regresión)', line=dict(color='red'))

# Añadir la tendencia polinómica
fig.add_scatter(x=ventas_por_mes_marca['DATE'], y=ventas_por_mes_marca['Tendencia_Polinomica'],
                mode='lines', name='Tendencia Polinómica', line=dict(color='green', dash='dot'))

# Mostrar el gráfico
fig.update_layout(title=f'Tendencia de Ventas en {marca_mas_ventas}',
                  xaxis_title='Fecha', yaxis_title='Cantidad de Productos Vendidos')
fig.show()




Estudio de la estacionalidad

In [19]:
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Obtener la marca con más ventas en los datos reales
marca_mas_ventas = df_actual.groupby('SUBBRAND')['AMOUNT'].sum().idxmax()

# Filtrar las ventas para la marca con más ventas
ventas_marca_mas = df_actual[df_actual['SUBBRAND'] == marca_mas_ventas].copy()

# Crear la columna `DATE` en el DataFrame para la agrupación por mes y año
ventas_marca_mas['DATE'] = pd.to_datetime(ventas_marca_mas[['YEAR', 'MONTH']].assign(day=1))

# Agrupar por fecha y sumar las ventas (AMOUNT) por cada fecha
ventas_por_mes_marca = ventas_marca_mas.groupby('DATE')['AMOUNT'].sum().reset_index()

# Estacionalidad: Gráfico de ventas por mes (promedio por mes)
ventas_por_mes_marca['Mes'] = ventas_por_mes_marca['DATE'].dt.month
ventas_estacionalidad_marca = ventas_por_mes_marca.groupby('Mes')['AMOUNT'].mean().reset_index()

# Crear gráfico de barras para estacionalidad
fig_estacionalidad_marca = px.bar(ventas_estacionalidad_marca, x='Mes', y='AMOUNT',
                                  title=f'Estacionalidad de Ventas para {marca_mas_ventas}',
                                  labels={'AMOUNT': 'Cantidad Promedio de Productos Vendidos', 'Mes': 'Mes'})
fig_estacionalidad_marca.update_layout(xaxis_title="Mes", yaxis_title="Ventas Promedio")

# Mostrar el gráfico de estacionalidad
fig_estacionalidad_marca.show()


Como los valores de ventas son muy grandes, las barras pueden parecer casi iguales, lo que hace difícil observar las diferencias. Para hacer un "zoom" en las diferencias, se va a aplicar una escala logarítmica al eje Y para centrarse en las diferencias más pequeñas (que siguen siendo de tamaño de millones de unidades).


In [20]:
import plotly.express as px
import pandas as pd

# Extraer el mes y calcular el promedio por cada mes
ventas_por_mes_marca['Mes'] = ventas_por_mes_marca['DATE'].dt.month
ventas_estacionalidad_marca = ventas_por_mes_marca.groupby('Mes')['AMOUNT'].mean().reset_index()

# Crear gráfico de barras para estacionalidad
fig_estacionalidad_marca = px.bar(ventas_estacionalidad_marca, x='Mes', y='AMOUNT',
                                  title=f'Estacionalidad de Ventas para {marca_mas_ventas}',
                                  labels={'AMOUNT': 'Cantidad Promedio de Productos Vendidos', 'Mes': 'Mes'})

# Añadir una línea para conectar los puntos de cada mes, resaltando estacionalidad
fig_estacionalidad_marca.add_scatter(x=ventas_estacionalidad_marca['Mes'], y=ventas_estacionalidad_marca['AMOUNT'],
                                     mode='lines+markers', name='Promedio Mensual', line=dict(color='blue'))

# Ajustar el eje Y a escala logarítmica para hacer un 'zoom' en las diferencias
fig_estacionalidad_marca.update_layout(
    xaxis_title="Mes",
    yaxis_title="Ventas Promedio",
    yaxis_type="log"  # Escala logarítmica para el eje Y
)

# Mostrar el gráfico de estacionalidad
fig_estacionalidad_marca.show()


Se observa un patrón repetitivo en las ventas: entre los meses 3 y 4 hay un descenso, seguido de una caída similar entre los meses 5, 6 y 7, que se repite en los meses 8, 9 y 10. Además, enero destaca por tener valores de ventas significativamente más bajos en comparación con el resto de los meses. Para el resto de los meses, no se identifica un comportamiento claro. Este patrón sugiere que las ventas experimentan fluctuaciones estacionales, con descensos en ciertas épocas del año y un inicio de año con ventas más bajas.

Cada 3 meses se llega a un mínimo (en comparación con el mes anterior).

Volvemos a mostrar las gráficas con todos los componentes juntos.

In [9]:
import pandas as pd
import plotly.express as px

# Filtrar solo las ventas reales
df_actuals = df[df['SCENARIO'] == 'actual']

# Identificar el país con menos ventas totales
country_sales = df_actuals.groupby('COUNTRY')['AMOUNT'].sum()
min_sales_country = country_sales.idxmin()

# Identificar la marca con más ventas totales
brand_sales = df_actuals.groupby('SUBBRAND')['AMOUNT'].sum()
max_sales_brand = brand_sales.idxmax()

# Filtrar datos de ventas para el país con menos ventas y la marca con más ventas
df_min_sales_country = df_actuals[df_actuals['COUNTRY'] == min_sales_country].copy()
df_max_sales_brand = df_actuals[df_actuals['SUBBRAND'] == max_sales_brand].copy()

# Convertir las columnas de año y mes a una sola columna de fecha
df_min_sales_country['DATE'] = pd.to_datetime(df_min_sales_country['YEAR'].astype(str) + '-' + df_min_sales_country['MONTH'].astype(str) + '-01')
df_max_sales_brand['DATE'] = pd.to_datetime(df_max_sales_brand['YEAR'].astype(str) + '-' + df_max_sales_brand['MONTH'].astype(str) + '-01')

# Agrupar las ventas mensuales para cada uno
df_min_sales_country_monthly = df_min_sales_country.groupby('DATE')['AMOUNT'].sum().reset_index()
df_max_sales_brand_monthly = df_max_sales_brand.groupby('DATE')['AMOUNT'].sum().reset_index()

# Calcular la tendencia utilizando un promedio móvil de 6 meses para suavizar
df_min_sales_country_monthly['Tendencia'] = df_min_sales_country_monthly['AMOUNT'].rolling(window=6, center=True).mean()
df_max_sales_brand_monthly['Tendencia'] = df_max_sales_brand_monthly['AMOUNT'].rolling(window=6, center=True).mean()

# Estimar la estacionalidad restando la tendencia de los valores originales y aplicando una media mensual
df_min_sales_country_monthly['Estacionalidad'] = df_min_sales_country_monthly['AMOUNT'] - df_min_sales_country_monthly['Tendencia']
df_max_sales_brand_monthly['Estacionalidad'] = df_max_sales_brand_monthly['AMOUNT'] - df_max_sales_brand_monthly['Tendencia']

# Crear gráficos con Plotly Express para el país y la marca, usando nombres personalizados para la leyenda
fig_min_country = px.line(df_min_sales_country_monthly, x='DATE', y=['AMOUNT', 'Tendencia', 'Estacionalidad'],
                          labels={'value': 'Ventas Totales', 'variable': 'Componente', 'AMOUNT': 'Datos'},
                          title=f"Tendencia y Estacionalidad de Ventas en {min_sales_country}")
fig_min_country.for_each_trace(lambda t: t.update(name={'AMOUNT': 'Datos', 'Tendencia': 'Tendencia', 'Estacionalidad': 'Estacionalidad'}[t.name]))
fig_min_country.update_traces(line=dict(width=2))
fig_min_country.show()

fig_max_brand = px.line(df_max_sales_brand_monthly, x='DATE', y=['AMOUNT', 'Tendencia', 'Estacionalidad'],
                        labels={'value': 'Ventas Totales', 'variable': 'Componente', 'AMOUNT': 'Datos'},
                        title=f"Tendencia y Estacionalidad de Ventas de la Marca {max_sales_brand}")
fig_max_brand.for_each_trace(lambda t: t.update(name={'AMOUNT': 'Datos', 'Tendencia': 'Tendencia', 'Estacionalidad': 'Estacionalidad'}[t.name]))
fig_max_brand.update_traces(line=dict(width=2))
fig_max_brand.show()




Volvemos a comprobar que efectivamente, no hay estacionalidad en las ventas de España, a no ser que fuera una estacionalidad anual con un pico de subida en los meses de verano. No tiene tendencia.

Por otro lado, las ventas de la marca Pepsi Max muestran una tendencia ascendente y una estacionalidad claras, con una caída en las ventas cada 3 meses aproximadamente.

# Pregunta 3

Cuales son las predicciones hechas en España y como de buenas son.


In [21]:
import pandas as pd
import plotly.graph_objects as go

# Desactivar el warning de "SettingWithCopyWarning" temporalmente
pd.options.mode.chained_assignment = None

# Filtrar solo los datos correspondientes a España
df_spain = df[df['COUNTRY'] == 'Spain']

# Separar las predicciones ('AI_forecast') y las ventas reales ('actual')
df_predictions = df_spain[df_spain['SCENARIO'] == 'AI_forecast']
df_actuals = df_spain[df_spain['SCENARIO'] == 'actual']

# Crear la columna 'DATE' para las predicciones (basada en YEAR y MONTH)
df_predictions['DATE'] = pd.to_datetime(df_predictions['YEAR'].astype(str) + '-' + df_predictions['MONTH'].astype(str).str.zfill(2) + '-01')

# Agrupar las predicciones por 'SUBBRAND' (marca) y 'DATE', calculando la media diaria de las predicciones
df_predictions_grouped = df_predictions.groupby(['SUBBRAND', 'DATE'], as_index=False)['AMOUNT'].mean()

# Crear gráficos de predicción para cada marca
for marca in df_predictions_grouped['SUBBRAND'].unique():
    # Filtrar los datos de la marca actual para las predicciones
    df_marca_predictions = df_predictions[df_predictions['SUBBRAND'] == marca].copy()  # Usar copy() para evitar el SettingWithCopyWarning

    # Crear una figura vacía
    fig = go.Figure()

    # Añadir la dispersión de las predicciones (puntos pequeños)
    fig.add_trace(go.Scatter(x=df_marca_predictions['DATE'],
                             y=df_marca_predictions['AMOUNT'],
                             mode='markers',
                             name='Predicciones',
                             marker=dict(size=4)))

    # Añadir la línea de predicción promediada para esta marca
    fig.add_trace(go.Scatter(x=df_predictions_grouped[df_predictions_grouped['SUBBRAND'] == marca]['DATE'],
                             y=df_predictions_grouped[df_predictions_grouped['SUBBRAND'] == marca]['AMOUNT'],
                             mode='lines',
                             name='Predicción Promediada',
                             line=dict(color='blue', width=3)))  # Línea azul más gruesa

    # Filtrar los datos reales de la marca actual
    df_marca_actuals = df_actuals[df_actuals['SUBBRAND'] == marca].copy()  # Usar copy() para evitar el SettingWithCopyWarning

    # Crear la columna 'REAL_DATE' para las ventas reales (también basada en YEAR y MONTH) usando .loc
    df_marca_actuals.loc[:, 'REAL_DATE'] = pd.to_datetime(df_marca_actuals['YEAR'].astype(str) + '-' + df_marca_actuals['MONTH'].astype(str).str.zfill(2) + '-01')

    # Ordenar los datos reales por fecha (REAL_DATE)
    df_marca_actuals = df_marca_actuals.sort_values(by='REAL_DATE')

    # Añadir la línea de ventas reales al gráfico (si existen datos reales)
    if not df_marca_actuals.empty:
        fig.add_trace(go.Scatter(x=df_marca_actuals['REAL_DATE'],
                                 y=df_marca_actuals['AMOUNT'],
                                 mode='lines',
                                 name='Ventas Reales',
                                 line=dict(color='black', width=3)))  # Línea negra más gruesa

    # Ajustar el layout para que sea más claro
    fig.update_layout(title=f'Predicciones de Ventas para {marca} en España',
                      xaxis_title='Fecha de Predicción',
                      yaxis_title='Cantidad de Productos Vendidos')

    # Mostrar el gráfico
    fig.show()


Como hay varias predicciones hechas en momentos diferentes, vamos a mostrar las ventas reales frente a las mejores predicciones.

In [23]:
import pandas as pd
import plotly.graph_objects as go

# Desactivar el warning de "SettingWithCopyWarning" temporalmente
pd.options.mode.chained_assignment = None

# Filtrar solo los datos correspondientes a España
df_spain = df[df['COUNTRY'] == 'Spain']

# Separar las predicciones ('AI_forecast') y las ventas reales ('actual')
df_predictions = df_spain[df_spain['SCENARIO'] == 'AI_forecast']
df_actuals = df_spain[df_spain['SCENARIO'] == 'actual']

# Crear la columna 'DATE' para las predicciones (basada en YEAR y MONTH)
df_predictions['DATE'] = pd.to_datetime(df_predictions['YEAR'].astype(str) + '-' + df_predictions['MONTH'].astype(str).str.zfill(2) + '-01')
df_actuals['REAL_DATE'] = pd.to_datetime(df_actuals['YEAR'].astype(str) + '-' + df_actuals['MONTH'].astype(str).str.zfill(2) + '-01')

# Merging para combinar predicciones y reales
df_merged = df_predictions.merge(
    df_actuals[['COUNTRY', 'SUBBRAND', 'YEAR', 'MONTH', 'AMOUNT', 'REAL_DATE']],
    on=['COUNTRY', 'SUBBRAND', 'YEAR', 'MONTH'],
    suffixes=('_forecast', '_actual')
)

# Calcular el error absoluto entre la predicción y el valor real
df_merged['ABS_ERROR'] = abs(df_merged['AMOUNT_forecast'] - df_merged['AMOUNT_actual'])

# Seleccionar la predicción con el menor error absoluto
best_predictions = df_merged.loc[df_merged.groupby(['COUNTRY', 'SUBBRAND', 'YEAR', 'MONTH'])['ABS_ERROR'].idxmin()]

# Crear gráficos de predicción para cada producto dentro de cada marca
for subbrand in best_predictions['SUBBRAND'].unique():
    # Filtrar y ordenar los datos reales del producto actual
    df_subbrand_actuals = df_actuals[df_actuals['SUBBRAND'] == subbrand].sort_values(by='REAL_DATE')
    df_subbrand_predictions = best_predictions[best_predictions['SUBBRAND'] == subbrand].sort_values(by='DATE')

    # Crear una figura vacía
    fig = go.Figure()

    # Añadir la línea de predicción seleccionada para este producto
    fig.add_trace(go.Scatter(x=df_subbrand_predictions['DATE'],
                             y=df_subbrand_predictions['AMOUNT_forecast'],
                             mode='lines',
                             name='Predicción Más Cercana',
                             line=dict(color='blue', width=3)))

    # Añadir la línea de ventas reales al gráfico
    fig.add_trace(go.Scatter(x=df_subbrand_actuals['REAL_DATE'],
                             y=df_subbrand_actuals['AMOUNT'],
                             mode='lines',
                             name='Ventas Reales',
                             line=dict(color='black', width=3)))

    # Ajustar el layout para que sea más claro
    fig.update_layout(title=f'Predicciones de Ventas para {subbrand} en España',
                      xaxis_title='Fecha de Predicción',
                      yaxis_title='Cantidad de Productos Vendidos')

    # Mostrar el gráfico
    fig.show()


Cómo de buenas son las predicciones

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# Filtrar solo los datos correspondientes a España
df_spain = df[df['COUNTRY'] == 'Spain']

# Separar las predicciones ('AI_forecast') y las ventas reales ('actual')
df_predictions = df_spain[df_spain['SCENARIO'] == 'AI_forecast']
df_actuals = df_spain[df_spain['SCENARIO'] == 'actual']

# Crear la columna 'DATE' para las predicciones (basada en YEAR y MONTH)
df_predictions['DATE'] = pd.to_datetime(df_predictions['YEAR'].astype(str) + '-' + df_predictions['MONTH'].astype(str).str.zfill(2) + '-01')

# Crear la columna 'REAL_DATE' para las ventas reales (también basada en YEAR y MONTH)
df_actuals['REAL_DATE'] = pd.to_datetime(df_actuals['YEAR'].astype(str) + '-' + df_actuals['MONTH'].astype(str).str.zfill(2) + '-01')

# Unir las predicciones con los valores reales por fecha (YEAR y MONTH)
df_comparison = pd.merge(df_predictions, df_actuals, how='left', left_on=['SUBBRAND', 'DATE'], right_on=['SUBBRAND', 'REAL_DATE'])

# Calcular el error absoluto entre las predicciones y los valores reales
df_comparison['ERROR_ABS'] = np.abs(df_comparison['AMOUNT_x'] - df_comparison['AMOUNT_y'])  # Predicción - Real en valor absoluto

# Agrupar por fecha de predicción (DATE) y calcular el promedio del error absoluto por fecha
df_avg_error = df_comparison.groupby(['DATE', 'SUBBRAND'], as_index=False)['ERROR_ABS'].mean()

# Crear gráfico de líneas mostrando el error absoluto promedio por fecha de predicción
fig = px.line(df_avg_error, x='DATE', y='ERROR_ABS', color='SUBBRAND',
              title="Error Absoluto Promedio en Función de la Fecha de Predicción",
              labels={'ERROR_ABS': 'Error Absoluto Promedio', 'DATE': 'Fecha de Predicción'},
              markers=True,
              color_discrete_map={
                  'Pepsi Max (L3)': 'blue',
                  'Pepsi Regular (L3)': 'red',
                  'Lipton (L3)': 'green',
                  '7up Free (L3)': 'purple',
                  '7up (L3)': 'orange'
              })

# Ajustar el rango del eje X para que termine en septiembre de 2024
fig.update_layout(
    xaxis_title="Fecha de Predicción",
    yaxis_title="Error Absoluto Promedio",
    showlegend=True,  # Para mostrar la leyenda que indica las marcas
    title_x=0.5,  # Centrar el título
    xaxis=dict(
        range=[df_avg_error['DATE'].min(), pd.to_datetime('2024-09-01')],  # Limitar el eje X hasta septiembre de 2024
    )
)

# Mostrar el gráfico
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# Filtrar solo los datos correspondientes a España
df_spain = df[df['COUNTRY'] == 'Spain']

# Separar las predicciones ('AI_forecast') y las ventas reales ('actual')
df_predictions = df_spain[df_spain['SCENARIO'] == 'AI_forecast']
df_actuals = df_spain[df_spain['SCENARIO'] == 'actual']

# Crear la columna 'DATE' para las predicciones (basada en YEAR y MONTH)
df_predictions['DATE'] = pd.to_datetime(df_predictions['YEAR'].astype(str) + '-' + df_predictions['MONTH'].astype(str).str.zfill(2) + '-01')

# Crear la columna 'REAL_DATE' para las ventas reales (también basada en YEAR y MONTH)
df_actuals['REAL_DATE'] = pd.to_datetime(df_actuals['YEAR'].astype(str) + '-' + df_actuals['MONTH'].astype(str).str.zfill(2) + '-01')

# Unir las predicciones con los valores reales por fecha (YEAR y MONTH)
df_comparison = pd.merge(df_predictions, df_actuals, how='left', left_on=['SUBBRAND', 'DATE'], right_on=['SUBBRAND', 'REAL_DATE'])

# Calcular la diferencia entre las predicciones y los valores reales
# Esto ya da el valor absoluto del error
df_comparison['ERROR_ABS'] = np.abs(df_comparison['AMOUNT_x'] - df_comparison['AMOUNT_y'])  # Predicción - Real en valor absoluto

# Graficar un histograma del error absoluto
fig = px.histogram(df_comparison, x='ERROR_ABS', nbins=30, title="Distribución del Error Absoluto entre Predicciones y Ventas Reales",
                   labels={'ERROR_ABS': 'Error Absoluto (Predicción - Real)'},
                   color_discrete_sequence=['blue'])

fig.update_layout(
    xaxis_title="Error Absoluto (Cantidad)",
    yaxis_title="Frecuencia",
    bargap=0.1
)

fig.show()

# Mostrar estadísticas de los errores
mae = df_comparison['ERROR_ABS'].mean()  # Error absoluto medio
mape = (df_comparison['ERROR_ABS'] / df_comparison['AMOUNT_y']).mean() * 100  # Error porcentual medio
rmse = np.sqrt((df_comparison['ERROR_ABS'] ** 2).mean())  # Raíz del error cuadrático medio

print(f"MAE (Error absoluto medio): {mae:.2f}")
print(f"MAPE (Error porcentual absoluto medio): {mape:.2f}%")
print(f"RMSE (Raíz del error cuadrático medio): {rmse:.2f}")



MAE (Error absoluto medio): 15741.40
MAPE (Error porcentual absoluto medio): 17.79%
RMSE (Raíz del error cuadrático medio): 23934.36


El gráfico muestra que la mayoría de los errores absolutos entre las predicciones y las ventas reales se encuentran en el rango de 5k a 15k, lo que indica que las predicciones son generalmente precisas. La mayor parte de los errores están por debajo de 30k, lo que refuerza la idea de que las predicciones son razonablemente cercanas a los valores reales. Además, es extremadamente raro observar errores superiores a los 60k, lo que sugiere que los casos de predicciones muy inexactas son poco frecuentes.

El MAE (Error Absoluto Medio) de 15,741.40 sugiere que, en promedio, el error entre las predicciones y las ventas reales es de aproximadamente 15,741 unidades, lo que concuerda con la observación de que la mayoría de los errores se encuentran en el rango de 5k a 15k.

El MAPE (Error Porcentual Absoluto Medio) de 17.79% indica que, en promedio, el error porcentual entre las predicciones y los valores reales es del 17.79%. Este porcentaje está alineado con la afirmación de que las predicciones son razonablemente precisas, aunque hay una variabilidad considerable.

Finalmente, el RMSE (Raíz del Error Cuadrático Medio) de 23,934.36 refuerza la idea de que, aunque hay algunos errores grandes, la mayoría no son extremos, ya que el RMSE no es excesivamente alto en comparación con el MAE. En general, la distribución del error muestra que la mayoría de los valores tienen errores inferiores a 30k, y es muy raro que el error exceda los 60k. Esto sugiere que, aunque las predicciones no son perfectas, son bastante consistentes y adecuadas para la mayoría de los casos.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd

# Filtrar solo los datos correspondientes a España
df_spain = df[df['COUNTRY'] == 'Spain']

# Separar las predicciones ('AI_forecast') y las ventas reales ('actual')
df_predictions = df_spain[df_spain['SCENARIO'] == 'AI_forecast']
df_actuals = df_spain[df_spain['SCENARIO'] == 'actual']

# Crear la columna 'DATE' para las predicciones (basada en YEAR y MONTH)
df_predictions['DATE'] = pd.to_datetime(df_predictions['YEAR'].astype(str) + '-' + df_predictions['MONTH'].astype(str).str.zfill(2) + '-01')

# Crear la columna 'REAL_DATE' para las ventas reales (también basada en YEAR y MONTH)
df_actuals['REAL_DATE'] = pd.to_datetime(df_actuals['YEAR'].astype(str) + '-' + df_actuals['MONTH'].astype(str).str.zfill(2) + '-01')

# Unir las predicciones con los valores reales por fecha (YEAR y MONTH)
df_comparison = pd.merge(df_predictions, df_actuals, how='left', left_on=['SUBBRAND', 'DATE'], right_on=['SUBBRAND', 'REAL_DATE'])

# Calcular la diferencia entre las predicciones y los valores reales en valor absoluto
df_comparison['ERROR_ABS'] = np.abs(df_comparison['AMOUNT_x'] - df_comparison['AMOUNT_y'])  # Predicción - Real en valor absoluto

# Crear una figura de subplots para mostrar todos los gráficos en un solo grid
n_subbrands = len(df_comparison['SUBBRAND'].unique())  # Número de marcas únicas
n_cols = 3  # Número de columnas
n_rows = int(np.ceil(n_subbrands / n_cols))  # Número de filas necesarias

# Verifica que el número de 'subplot_titles' coincida con la cantidad de marcas
subplot_titles = df_comparison['SUBBRAND'].unique()

fig = make_subplots(
    rows=n_rows, cols=n_cols,
    subplot_titles=subplot_titles,  # Títulos de cada subgrafico
    shared_xaxes=False,  # Para no compartir el eje X entre los subgráficos
    shared_yaxes=False,  # Para no compartir el eje Y entre los subgráficos
    vertical_spacing=0.2,  # Aumenta el espacio vertical entre las filas
    horizontal_spacing=0.1  # Aumenta el espacio horizontal entre las columnas
)

# Crear los histogramas para cada 'SUBBRAND'
for i, marca in enumerate(df_comparison['SUBBRAND'].unique()):
    df_marca = df_comparison[df_comparison['SUBBRAND'] == marca]

    # Calcular los histogramas
    hist_data = df_marca['ERROR_ABS']

    # Calcular la fila y columna de los subgráficos
    row = i // n_cols + 1  # Fila en la que se va a colocar el gráfico
    col = i % n_cols + 1   # Columna en la que se va a colocar el gráfico

    # Agregar el histograma al subplot sin normalizar para mostrar frecuencias absolutas
    fig.add_trace(
        go.Histogram(
            x=hist_data,
            nbinsx=30,  # Número de bins
            name=marca,
            opacity=0.7,
            histnorm='',  # Frecuencias absolutas (sin normalizar)
        ),
        row=row, col=col  # Especificar la fila y columna del subplot
    )

    # Actualizar los ejes de cada subgráfico
    fig.update_xaxes(
        title_text="Error Absoluto",  # Etiqueta del eje X
        row=row, col=col              # Aplicar a la fila y columna correspondiente
    )
    fig.update_yaxes(
        title_text="Frecuencia",  # Etiqueta del eje Y
        row=row, col=col          # Aplicar a la fila y columna correspondiente
    )

# Actualizar el layout del gráfico
fig.update_layout(
    height=400 * n_rows,  # Establecer la altura total dependiendo del número de filas
    width=1200,           # Establecer el ancho total (más ancho)
    title_text='Distribución del Error Absoluto por Marca',
    showlegend=False,  # Desactivar la leyenda general
    title_x=0.5,  # Centrar el título del gráfico
    margin=dict(t=100, b=100, l=50, r=50)  # Ajustar márgenes para más espacio en los bordes
)

# Mostrar el gráfico
fig.show()


Esto es para ver las distribuciones del error en las predicciones.

De los gráficos anteriores podemos concluir que:

- Lipton muestra los errores más pequeños y las frecuencias más bajas, lo que parece indicar que las predicciones para esta marca son las más precisas en términos del error absoluto.
- En contraste, Pepsi Regular presenta los errores más grandes, con algunos casos superando los 150k en el error absoluto, y también muestra las frecuencias más altas, lo que sugiere que las predicciones para esta marca son menos exactas y más variables.

Vamos a hacer un análisis detallado de las métricas de error, estudiando cuatro indicadores clave: MAE, MSE, RMSE y MAPE.

* MAE (Error Absoluto Medio): Mide la magnitud promedio de los errores en las
predicciones, sin considerar su dirección. Es útil para tener una idea clara de cuánto se desvían las predicciones en promedio de los valores reales.

* MSE (Error Cuadrático Medio): Calcula el promedio de los cuadrados de los errores. Penaliza más los errores grandes, lo que lo hace más sensible a los valores atípicos. Este valor tiende a ser más grande que el MAE si hay errores grandes.

* RMSE (Raíz del Error Cuadrático Medio): Es la raíz cuadrada del MSE, lo que la convierte en una métrica que está en la misma unidad que los datos originales, facilitando la interpretación. Al igual que el MSE, penaliza los errores grandes.

* MAPE (Error Absoluto Porcentual Medio): Mide la precisión de las predicciones en términos porcentuales. Ayuda a entender el error en relación al tamaño de las ventas reales. Es especialmente útil cuando los datos tienen escalas muy diferentes entre sí.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Inicializar una lista para almacenar los resultados de las métricas
metrics = []

# Asignar colores específicos en el orden deseado (azul, rojo, verde, morado, naranja)
specific_colors = ['#1f77b4', '#e31a1c', '#2ca02c', '#9467bd', '#ff7f0e']  # Azul, Rojo, Verde, Morado, Naranja
marca_colors = {marca: specific_colors[i % len(specific_colors)] for i, marca in enumerate(df_comparison['SUBBRAND'].unique())}

# Calcular las métricas para cada marca
for marca in df_comparison['SUBBRAND'].unique():
    df_marca = df_comparison[df_comparison['SUBBRAND'] == marca]

    # Eliminar filas con valores NaN en las predicciones o valores reales
    df_marca_clean = df_marca.dropna(subset=['AMOUNT_x', 'AMOUNT_y'])

    # Calcular las métricas
    y_true = df_marca_clean['AMOUNT_y']  # Ventas reales
    y_pred = df_marca_clean['AMOUNT_x']  # Predicciones

    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # en porcentaje

    # Almacenar los resultados
    metrics.append({
        'Marca': marca,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape
    })

# Convertir los resultados en un DataFrame
metrics_df = pd.DataFrame(metrics)

# Crear la figura de subgráficos
fig = make_subplots(
    rows=1, cols=4,  # Un solo row con 4 columnas
    subplot_titles=['MAE', 'MSE', 'RMSE', 'MAPE'],  # Títulos de cada subgráfico
)

# Agregar las barras de cada métrica a cada subgráfico
for i, marca in enumerate(metrics_df['Marca']):
    # Datos para cada métrica
    mae = metrics_df.loc[metrics_df['Marca'] == marca, 'MAE'].values[0]
    mse = metrics_df.loc[metrics_df['Marca'] == marca, 'MSE'].values[0]
    rmse = metrics_df.loc[metrics_df['Marca'] == marca, 'RMSE'].values[0]
    mape = metrics_df.loc[metrics_df['Marca'] == marca, 'MAPE'].values[0]

    # Obtener el color para la marca
    color = marca_colors[marca]

    # Agregar barras de cada métrica
    fig.add_trace(go.Bar(
        x=[marca], y=[mae],
        name='MAE',
        marker=dict(color=color),
    ), row=1, col=1)

    fig.add_trace(go.Bar(
        x=[marca], y=[mse],
        name='MSE',
        marker=dict(color=color),
    ), row=1, col=2)

    fig.add_trace(go.Bar(
        x=[marca], y=[rmse],
        name='RMSE',
        marker=dict(color=color),
    ), row=1, col=3)

    fig.add_trace(go.Bar(
        x=[marca], y=[mape],
        name='MAPE',
        marker=dict(color=color),
    ), row=1, col=4)

# Actualizar el layout de la figura
fig.update_layout(
    height=600,  # Establecer la altura total de la figura
    width=1200,  # Establecer el ancho total de la figura
    title_text="Métricas de Predicción por Marca",
    showlegend=False,  # Desactivar la leyenda general
    title_x=0.5,  # Centrar el título
)

# Mostrar el gráfico
fig.show()



Las predicciones para 7up Free (L3) son las más precisas, con errores relativamente bajos, tanto absolutos como relativos. En cambio, Pepsi Regular (L3) muestra los errores más altos, lo que indica una menor exactitud en las predicciones para esta marca. Lipton (L3), aunque tiene un error absoluto bajo, presenta una mayor variabilidad relativa en las predicciones.

Las peores predicciones son las de Lipton porque tienen el mayor error porcentual. En proporción, Lipton es el que tiene las peores predicciones y 7up Free el que mejor.

In [26]:
import pandas as pd
import plotly.express as px

# Desactivar el warning de "SettingWithCopyWarning" temporalmente
pd.options.mode.chained_assignment = None

# Filtrar solo los datos correspondientes a España
df_spain = df[df['COUNTRY'] == 'Spain']

# Separar las predicciones ('AI_forecast') y las ventas reales ('actual')
df_predictions = df_spain[df_spain['SCENARIO'] == 'AI_forecast']
df_actuals = df_spain[df_spain['SCENARIO'] == 'actual']

# Crear la columna 'DATE' para las predicciones y ventas reales
df_predictions['DATE'] = pd.to_datetime(df_predictions['YEAR'].astype(str) + '-' + df_predictions['MONTH'].astype(str).str.zfill(2) + '-01')
df_actuals['REAL_DATE'] = pd.to_datetime(df_actuals['YEAR'].astype(str) + '-' + df_actuals['MONTH'].astype(str).str.zfill(2) + '-01')

# Merging para combinar predicciones y reales
df_merged = df_predictions.merge(
    df_actuals[['COUNTRY', 'SUBBRAND', 'YEAR', 'MONTH', 'AMOUNT', 'REAL_DATE']],
    on=['COUNTRY', 'SUBBRAND', 'YEAR', 'MONTH'],
    suffixes=('_forecast', '_actual')
)

# Calcular el error absoluto y el error relativo para cada punto de predicción
df_merged['ABS_ERROR'] = abs(df_merged['AMOUNT_forecast'] - df_merged['AMOUNT_actual'])
df_merged['REL_ERROR'] = df_merged['ABS_ERROR'] / df_merged['AMOUNT_actual']

# Calcular el MAE (Error Medio Absoluto) y el error relativo medio por producto
metrics_by_product = df_merged.groupby('SUBBRAND').agg(
    MAE=('ABS_ERROR', 'mean'),
    Mean_Relative_Error=('REL_ERROR', 'mean')
).reset_index()

# Gráfico 1: Error medio absoluto (MAE) por producto
fig_mae = px.bar(metrics_by_product, x='SUBBRAND', y='MAE',
                 title="Error Medio Absoluto (MAE) por Producto",
                 labels={'MAE': 'Error Medio Absoluto', 'SUBBRAND': 'Producto'})
fig_mae.show()

# Gráfico 2: Error relativo medio por producto
fig_precision = px.bar(metrics_by_product, x='SUBBRAND', y='Mean_Relative_Error',
                       title="Error Relativo Medio por Producto",
                       labels={'Mean_Relative_Error': 'Error Relativo Medio', 'SUBBRAND': 'Producto'})
fig_precision.show()



Observando el error relativo medio, la marca Lipton tiene las peores predicciones y la marca 7up Free las mejores.